# Multimodal RAG with Gemini 2.0
By: Wilfredo Aaron Sosa Ramos (AI Lab Manager)

In [ ]:
!pip install -q langchain langchain_core langchain_community langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
!pip install -q pypdf beautifulsoup4 arxiv pymupdf assemblyai

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.8 MB/s eta 0:00:00


In [3]:
!pip install -q chroma langchain_chroma

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

##Set env. variables

In [4]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['ASSEMBLYAI_API_KEY'] = userdata.get('ASSEMBLYAI_API_KEY')

##Class for downloading files

In [27]:
import tempfile
import uuid
import requests

class FileHandler:
    def __init__(self, file_loader, file_extension):
        self.file_loader = file_loader
        self.file_extension = file_extension

    def load(self, url):
        # Generate a unique filename with a UUID prefix
        unique_filename = f"{uuid.uuid4()}.{self.file_extension}"

        try:
            # Download the file from the URL and save it to a temporary file
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # Raise an HTTPError for bad responses

            with tempfile.NamedTemporaryFile(delete=False, prefix=unique_filename) as temp_file:
                temp_file.write(response.content)
                temp_file_path = temp_file.name

        except requests.exceptions.RequestException as req_err:
            raise Exception(f"Failed to download file from URL", url) from req_err
        except Exception as e:
            raise Exception(f"Failed to handle file download", url) from e

        # Use the file_loader to load the documents
        try:
            loader = self.file_loader(file_path=temp_file_path)
        except Exception as e:
            raise Exception(f"No file found", temp_file_path) from e

        try:
            documents = loader.load()
        except Exception as e:
            raise Exception(f"No file content available", temp_file_path) from e

        # Remove the temporary file
        os.remove(temp_file_path)

        return documents

##1. PDF Document Loader:

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

def load_pdf_documents(pdf_url: str):
    pdf_loader = FileHandler(PyPDFLoader, "pdf")
    docs = pdf_loader.load(pdf_url)

    if docs:
        split_docs = splitter.split_documents(docs)

        return split_docs

In [7]:
docs_pdf = load_pdf_documents("https://mackinstitute.wharton.upenn.edu/wp-content/uploads/2023/08/LLM-Ideas-Working-Paper.pdf")

In [8]:
docs_pdf

[Document(metadata={'source': '/tmp/9db6edb7-b04f-43c4-a606-5e3a334e5b9f.pdf_vlij2ca', 'page': 0}, page_content='Co-Brand Name \nIdeas Are Dimes A Dozen: Large Language Models For Idea \nGeneration In Innovation \nKaran Girotra, Lennart Meincke, Christian Terwiesch, and Karl T. Ulrich1\nJuly 10, 2023 \nMack Institute for Innovation Management, The Wharton School, University of Pennsylvania \nCornell Tech and Johnson College of Business, Cornell University\nAbstract \nLarge language models (LLMs) such as OpenID’s GPT series have shown remarkable capabilities in generating \nfluent and coherent text in various domains. We compare the ideation capabilities of ChatGPT-4, a chatbot based \non a state-of-the-art LLM, with those of students at an elite university. ChatGPT-4 can generate ideas much faster \nand cheaper than students, and the ideas are on average of higher quality (as measured by purchase-intent \nsurveys) and exhibit higher variance in quality. More important, the vast majorit

#2. Web URL Loader:

In [11]:
from langchain_community.document_loaders import WebBaseLoader

def load_url_documents(url: str):
    web_url_loader = WebBaseLoader(url)
    docs = web_url_loader.load()

    if docs:
        split_docs = splitter.split_documents(docs)

        return split_docs

In [12]:
url_docs = load_url_documents("https://en.wikipedia.org/wiki/Large_language_model")

In [13]:
url_docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Large_language_model', 'title': 'Large language model - Wikipedia', 'language': 'en'}, page_content='Large language model - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\

##3. Arxiv:

In [14]:
from langchain_community.document_loaders import ArxivLoader

def load_arxiv_documents(query: str):
    arxiv_loader = ArxivLoader(
        query=query,
        load_max_docs=2,
    )
    docs = arxiv_loader.load()

    if docs:
        split_docs = splitter.split_documents(docs)

        return split_docs

In [15]:
arxiv_docs = load_arxiv_documents("Large Language Models")

In [16]:
arxiv_docs

[Document(metadata={'Published': '2023-06-12', 'Title': 'Lost in Translation: Large Language Models in Non-English Content Analysis', 'Authors': 'Gabriel Nicholas, Aliya Bhatia', 'Summary': "In recent years, large language models (e.g., Open AI's GPT-4, Meta's LLaMa,\nGoogle's PaLM) have become the dominant approach for building AI systems to\nanalyze and generate language online. However, the automated systems that\nincreasingly mediate our interactions online -- such as chatbots, content\nmoderation systems, and search engines -- are primarily designed for and work\nfar more effectively in English than in the world's other 7,000 languages.\nRecently, researchers and technology companies have attempted to extend the\ncapabilities of large language models into languages other than English by\nbuilding what are called multilingual language models.\n  In this paper, we explain how these multilingual language models work and\nexplore their capabilities and limits. Part I provides a simple

##4. Images:

In [35]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm_for_img = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [36]:
from langchain_core.messages import HumanMessage
from langchain_core.documents import Document

def generate_docs_from_img(img_url):
    message = HumanMessage(
    content=[
            {
                "type": "text",
                "text": "Give me a summary of what you see in the image. It must be 3 detailed paragraphs.",
            },
            {"type": "image_url", "image_url": img_url},
        ]
    )

    try:
        response = llm_for_img.invoke([message]).content
        print(f"Generated summary: {response}")
        docs = Document(page_content=response, metadata={"source": img_url})
        split_docs = splitter.split_documents([docs])
    except Exception as e:
        print(f"Error processing the request due to Invalid Content or Invalid Image URL")
        raise e

    return split_docs

In [37]:
img_docs = generate_docs_from_img("https://datasciencedojo.com/wp-content/uploads/Architecture-of-transformer-models-1030x579.jpg")

Generated summary: Here's a summary of the provided image in three detailed paragraphs:

The image diagrams the architecture of a Transformer, a neural network architecture commonly used in natural language processing.  The core of the architecture is built around the concept of "multi-head attention," which is visually represented as a central block.  This block takes input embeddings (processed representations of words or sub-words) and applies a series of linear transformations to create query (Q), key (K), and value (V) matrices. These matrices are then fed into a "scaled dot-product attention" mechanism,  a detailed breakdown of which is shown separately. The multi-head attention mechanism allows the model to weigh the importance of different parts of the input sequence when processing each word, capturing relationships between words across the sequence.  The output of the multi-head attention is then passed through an "add & norm" layer (representing addition and normalization op

In [38]:
img_docs

[Document(metadata={'source': 'https://datasciencedojo.com/wp-content/uploads/Architecture-of-transformer-models-1030x579.jpg'}, page_content="Here's a summary of the provided image in three detailed paragraphs:"),
 Document(metadata={'source': 'https://datasciencedojo.com/wp-content/uploads/Architecture-of-transformer-models-1030x579.jpg'}, page_content='The image diagrams the architecture of a Transformer, a neural network architecture commonly used in natural language processing.  The core of the architecture is built around the concept of "multi-head attention," which is visually represented as a central block.  This block takes input embeddings (processed representations of words or sub-words) and applies a series of linear transformations to create query (Q), key (K), and value (V) matrices. These matrices are then fed into a "scaled dot-product attention" mechanism,  a detailed breakdown of which is shown separately. The multi-head attention mechanism allows the model to weigh t

##5. Audio Document Loader (AssemblyAI)

In [39]:
from langchain_community.document_loaders import AssemblyAIAudioTranscriptLoader
def load_audio_documents(path: str):
    audio_loader = AssemblyAIAudioTranscriptLoader(file_path=path)

    docs = audio_loader.load()

    if docs:
        split_docs = splitter.split_documents(docs)

        return split_docs

In [40]:
audio_docs = load_audio_documents("/content/RAG for LLMs explained in 3 minutes.mp3")

In [41]:
audio_docs

[Document(metadata={'language_code': 'en_us', 'audio_url': 'https://cdn.assemblyai.com/upload/1bfc74e3-2be2-4d6d-85e0-cee0d2c818d1', 'punctuate': True, 'format_text': True, 'dual_channel': None, 'multichannel': None, 'audio_channels': None, 'webhook_url': None, 'webhook_auth_header_name': None, 'webhook_auth_header_value': None, 'audio_start_from': None, 'audio_end_at': None, 'word_boost': [], 'boost_param': None, 'filter_profanity': False, 'redact_pii': False, 'redact_pii_audio': False, 'redact_pii_audio_quality': None, 'redact_pii_policies': None, 'redact_pii_sub': None, 'speaker_labels': False, 'speakers_expected': None, 'content_safety': False, 'content_safety_confidence': None, 'iab_categories': False, 'custom_spelling': None, 'disfluencies': False, 'sentiment_analysis': False, 'auto_chapters': False, 'entity_detection': False, 'summarization': False, 'summary_model': None, 'summary_type': None, 'auto_highlights': False, 'language_detection': False, 'language_confidence_threshold'

##RAG Pipeline:

In [42]:
docs = docs_pdf + url_docs + arxiv_docs + img_docs + audio_docs

In [43]:
docs

[Document(metadata={'source': '/tmp/9db6edb7-b04f-43c4-a606-5e3a334e5b9f.pdf_vlij2ca', 'page': 0}, page_content='Co-Brand Name \nIdeas Are Dimes A Dozen: Large Language Models For Idea \nGeneration In Innovation \nKaran Girotra, Lennart Meincke, Christian Terwiesch, and Karl T. Ulrich1\nJuly 10, 2023 \nMack Institute for Innovation Management, The Wharton School, University of Pennsylvania \nCornell Tech and Johnson College of Business, Cornell University\nAbstract \nLarge language models (LLMs) such as OpenID’s GPT series have shown remarkable capabilities in generating \nfluent and coherent text in various domains. We compare the ideation capabilities of ChatGPT-4, a chatbot based \non a state-of-the-art LLM, with those of students at an elite university. ChatGPT-4 can generate ideas much faster \nand cheaper than students, and the ideas are on average of higher quality (as measured by purchase-intent \nsurveys) and exhibit higher variance in quality. More important, the vast majorit

In [45]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')

In [51]:
from langchain_chroma import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata

vector_store = Chroma(
    collection_name="rag_llm",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

In [52]:
vector_store.from_documents(filter_complex_metadata(docs), embeddings)

In [53]:
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 1, "fetch_k": 5}
)

In [75]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

runner = RunnableParallel(
                {"context": retriever,
                "attribute_collection": RunnablePassthrough()
                }
            )

In [80]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
    You are a knowledgeable expert in the field of Large Language Models (LLMs) with a focus on delivering accurate, professional, and context-aware responses. Use the provided context and question to craft a clear and concise answer in the specified language. Ensure your response is tailored to the given audience, considering technical accuracy and professional tone.

    - **Context**: {context}
    - **Question and Response Language**: {attribute_collection}

    **Guidelines**:
    1. Provide an accurate and professional response based on the given context.
    2. If applicable, include relevant examples or technical details to enhance understanding.
    3. Write the response in the specified language, maintaining clarity and professionalism.
    4. Keep the tone neutral and the content informative.

    **Output**:
    A detailed, contextually relevant answer to the question in the specified language.
    """,
    input_variables=["attribute_collection"]
)

In [77]:
from langchain_google_genai import GoogleGenerativeAI

model = GoogleGenerativeAI(model="gemini-2.0-flash-exp")

In [81]:
from langchain_core.output_parsers.string import StrOutputParser
chain = runner | prompt | model | StrOutputParser()

In [82]:
question = "What is a Large Language Model?"
lang = "en"

result = chain.invoke(
    f"""
    Question: {question}
    Response Language: {lang}
    """
)

In [83]:
result

'A Large Language Model (LLM) is a type of artificial intelligence model, specifically a deep learning model, that is trained on a massive dataset of text and code. These models utilize neural networks with a large number of parameters (hence "large") to learn complex patterns and relationships within the data. This enables them to perform a wide range of natural language processing (NLP) tasks, such as:\n\n*   **Text Generation:** Creating human-quality text, including articles, stories, poems, and code.\n*   **Text Summarization:** Condensing large amounts of text into shorter, more concise summaries.\n*   **Question Answering:** Answering questions based on provided text or general knowledge.\n*   **Text Translation:** Converting text from one language to another.\n*   **Sentiment Analysis:** Determining the emotional tone of a piece of text.\n*   **Code Generation:** Writing code in various programming languages based on descriptions.\n\nLLMs are typically based on the transformer 

In [84]:
question = "How can a LLM support in business?"
lang = "es"

result = chain.invoke(
    f"""
    Question: {question}
    Response Language: {lang}
    """
)

In [85]:
print(result)

Claro, aquí tienes una respuesta sobre cómo un LLM puede apoyar a las empresas, en español:

Los Modelos de Lenguaje Grandes (LLMs) ofrecen una amplia gama de aplicaciones que pueden impulsar la eficiencia, la innovación y la toma de decisiones en diversos aspectos de un negocio. Aquí te presento algunas formas clave en las que un LLM puede brindar soporte:

**1. Automatización de Tareas y Optimización de Procesos:**

*   **Atención al Cliente:** Los LLMs pueden potenciar chatbots y asistentes virtuales para responder preguntas frecuentes, gestionar solicitudes y proporcionar soporte 24/7, liberando a los agentes humanos para tareas más complejas. Pueden entender el lenguaje natural y proporcionar respuestas relevantes y personalizadas.
*   **Generación de Contenido:** Pueden crear contenido de marketing, descripciones de productos, publicaciones en redes sociales, correos electrónicos y otros tipos de texto, ahorrando tiempo y recursos a los equipos de marketing y comunicación.
*   **

In [99]:
os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY']=userdata.get('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT']='default'

In [100]:
question = "Why is Transformer Architecture important for LLMs?"
lang = "es"

result = chain.invoke(
    f"""
    Question: {question}
    Response Language: {lang}
    """
)

In [87]:
print(result)

La arquitectura Transformer es fundamental para los Modelos de Lenguaje Grandes (LLMs) por varias razones clave:

1. **Atención (Attention):** A diferencia de las arquitecturas recurrentes (RNNs) que procesan la secuencia de entrada de forma secuencial, los Transformers utilizan un mecanismo de atención que permite al modelo ponderar la importancia de diferentes palabras en la secuencia al procesar otra palabra. Esto significa que el modelo puede capturar relaciones a larga distancia entre palabras, algo que era difícil con las RNNs. Por ejemplo, en la frase "El perro que corrió rápido cruzó la calle", el mecanismo de atención permitiría al modelo entender que "corrió" se refiere al "perro" a pesar de que están separados en la oración.

2. **Paralelización:** La arquitectura Transformer permite el procesamiento paralelo de la secuencia de entrada. Dado que el mecanismo de atención opera simultáneamente sobre todas las posiciones de la secuencia, el entrenamiento puede ser mucho más ráp